In [1]:
from meta import *
from utils import *

# Load the audio

In [21]:
input_target_pairs = get_input_target_fname_pairs(GUITAR, OVERDRIVE)

instrument = GUITAR
fx = DISTORTION

audio_in = load_audio(os.path.join(params_path[instrument][NO_FX], input_target_pairs[1][0]))
audio_target = load_audio(os.path.join(params_path[instrument][OVERDRIVE], input_target_pairs[55][1]))
                          
print(audio_in.shape)
print(audio_target.shape)

loading: /import/c4dm-04/cv/fxdataset/guitar/NoFX/G93-56401-1111-21179.wav
loading: /import/c4dm-04/cv/fxdataset/guitar/Overdrive/G91-50300-4422-40431.wav
(32001, 1)
(32001, 1)
